In [ ]:
# -*- coding: utf-8 -*-

# Installs

In [ ]:
!pip install pyLDAvis

In [ ]:
!pip install bnlp_toolkit

In [ ]:
!pip install bnltk

In [ ]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git

  Cloning https://github.com/banglakit/bengali-stemmer.git to /tmp/pip-req-build-dkpm91mb
  Running command git clone -q https://github.com/banglakit/bengali-stemmer.git /tmp/pip-req-build-dkpm91mb
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6408 sha256=d1b2669a404996897162bc5c58ea4de6cdb03ba86da3bbfcfecf846bede8ca5e
  Stored in directory: /tmp/pip-ephem-wheel-cache-_5oh96k0/wheels/a1/ad/a1/4ba354059b17c00600a14e13a504e7bdb49f20f2f4e2f3639c
Successfully built bengali-stemmer


# Imports

In [ ]:
import pandas as pd
import numpy as np
import random
import re
import os

from sklearn.feature_extraction.text import CountVectorizer

from bnlp.corpus import stopwords as stopwords_1, punctuations, digits
from bnlp.corpus.util import remove_stopwords

from bengali_stemmer.rafikamal2014 import RafiStemmer
from bnltk.tokenize import Tokenizers

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Connect Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/My Drive/THESIS/LDA'

!pip install cache-magic
import cache_magic
!mkdir .cache
!ln -s '/content/drive/My Drive/THESIS/LDA/.cache' /content/.cache

/content/drive/My Drive/THESIS/LDA
mkdir: cannot create directory ‘.cache’: File exists
ln: failed to create symbolic link '/content/.cache/.cache': File exists


# Get Dataset

In [ ]:
articles = pd.read_csv('/content/drive/MyDrive/THESIS/Final Dataset/all_final_reduced.csv', error_bad_lines=False)
print(articles.isnull().sum())
articles

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Full_Text    0
Headline     0
Date         0
Topic        0
Source       0
Year         0
dtype: int64


,Full_Text,Headline,Date,Topic,Source,Year
0,"তিনি বলেছেন, “মানুষের যে ভোটের অধিকার সেটা হচ...",সংবিধানের দোহাই দিয়ে জনগণকে ‘অধিকারহীন’ করে র...,০৬-০১-২০১৮,রাজনীতি,bdnews24,২০১৮
1,শিবচর থানার পরিদর্শক (তদন্ত) শাজাহান মিয়া জান...,মাদারীপুরের শিবপুর উপজেলায় এক গৃহবধূকে ‘মেরে ...,০৯-০৮-২০১৮,বাংলাদেশ,bdnews24,২০১৮
2,আগামী ১৩ ও ১৪ মে কানাডার টরন্টো প্যাভিলিয়নে এ...,কানাডা প্রবাসী বাংলাদেশিদের আয়োজনে তৃতীয়বারের...,০৩-০৪-২০১৭,প্রবাস,bdnews24,২০১৭
3,দুটি যুব বিশ্বকাপে বাংলাদেশ দলের অধিনায়ক ছিলে...,২০১৬ যুব বিশ্বকাপে শেষ কোনো দলকে নেতৃত্ব দিয়ে...,০৫-০১-২০১৯,খেলা,bdnews24,২০১৯
4,"'হ্যাঁ, অনেক আগে আমি গাঁজা সেবন করতাম। কিন্তু ...",গাঁজা সেবনের কথা স্বীকার করলেন নিউজিল্যান্ডের ...,১-১০-২০২০,আন্তর্জাতিক,kalerkantha,২০২০
...,...,...,...,...,...,...
373305,নারায়ণগঞ্জের বন্দরে একটি ভবনের সেপটিক ট্যাংক ব...,সেপটিক ট্যাংক বিস্ফোরণে আহত অন্তঃসত্ত্বা নারীর...,০৮-০৫-২০২০,বাংলাদেশ,jugantor,২০২০
373306,হেফাজতে ইসলামের কেন্দ্রীয় যুগ্ম মহাসচিব ও ঢাকা...,জোশের কারণে ওয়াজ মাহফিলে মানহানিকর বক্তব্য দি...,২০-০৪-২০২১,রাজনীতি,jugantor,২০২১
373307,সোমবার সকাল ৬টার দিকে রাজৈর উপজেলার বোলগ্রামে...,মাদারীপুরের রাজৈরে একটি পিকআপ ভ্যান খাদে পড়ে ...,২৬-১১-২০১৮,বাংলাদেশ,bdnews24,২০১৮
373308,জেলা শহরের পাটপট্রি পূজা মন্দির কমিটির লোকজন ...,করোনাভাইরাস সংক্রমণের মহামারীর মধ্যে হিন্দু স...,২৯-০৩-২০২০,বাংলাদেশ,bdnews24,২০২০


In [ ]:
'''#drop Summary Field
articles.drop('Summary', inplace=True, axis=1)
articles.reset_index(drop=True)
articles'''
articles.dropna(inplace=True)
articles.reset_index(drop=True, inplace=True)
articles.isnull().sum()

Full_Text    0
Headline     0
Date         0
Topic        0
Source       0
Year         0
dtype: int64

In [ ]:
random.seed(42)
bangla_news_list = articles.Full_Text.to_list()
print(bangla_news_list[31])
random.shuffle(bangla_news_list)
print(bangla_news_list[31])

প্রধানমন্ত্রী শেখ হাসিনা দক্ষিণ এশিয়ার জন্য গ্লোবাল সেন্টার অন অ্যাডাপ্টেশন (জিসিএ) বাংলাদেশ আঞ্চলিক কার্যালয় আনুষ্ঠানিক উদ্বোধন করবেন।মঙ্গলবার বিকাল সাড়ে ৩টায় এই অনুষ্ঠানে অভিযোজন সম্পর্কিত গ্লোবাল সেন্টার বোর্ডের চেয়ারম্যান এবং জাতিসংঘের সাবেক মহাসচিব বান কি মুন উপস্থিত থাকবেন।দক্ষিণ এশীয় দেশগুলোর মন্ত্রীদের পাশাপাশি গ্লোবাল সেন্টার অন অ্যাডাপ্টেশনের প্রধান নির্বাহী কর্মকর্তা প্যাট্রিক ভারকুইজেন বক্তব্য রাখবেন বলেও পরিবেশ, বন ও জলবায়ু পরিবর্তন বিষয়ক মন্ত্রণালয়ের সংবাদ বিজ্ঞপ্তিতে জানিয়েছেন।আঞ্চলিক গ্লোবাল সেন্টার অন অ্যাডাপ্টেশন বাংলাদেশ’র ভার্চুয়াল উদ্বোধন অনুষ্ঠানের পর পরিবেশ, বন ও জলবায়ু পরিবর্তন বিষয়ক মন্ত্রণালয় একটি সাংবাদিক সম্মেলনের আয়োজন করবে। এতে পররাষ্ট্রমন্ত্রী ড. এ. কে আবদুল মোমেন এবং পরিবেশ, বন ও জলবায়ু পরিবর্তন বিষয়ক মন্ত্রী শাহাব উদ্দিন বক্তব্য রাখবেন।পরে দক্ষিণ এশিয়ার বিশেষজ্ঞ, শিক্ষাবিদ এবং জলবায়ু পরিবর্তন নিয়ে কর্মরত বৈশ্বিক সংস্থার প্রতিনিধিরা জিসিএ সাউথ এশিয়া পার্টনারশিপ ফোরামে অংশ নেবেন।প্যানেল আলোচনার পর ক্লাইমেট ভালনারেবল ফোরামের থিমেটিক অ্যাম্বেসেডর সায়মা ওয়াজ

# Pre-process

In [ ]:
stopwords_2 = "অবশ্য অনেক অনেকে অনেকেই অন্তত অথবা অথচ অর্থাত অন্য আজ আছে আপনার আপনি আবার আমরা আমাকে আমাদের আমার আমি আরও আর আগে আগেই আই অতএব আগামী অবধি অনুযায়ী আদ্যভাগে এই একই একে একটি এখন এখনও এখানে এখানেই এটি এটা এটাই এতটাই এবং একবার এবার এদের এঁদের এমন এমনকী এল এর এরা এঁরা এস এত এতে এসে একে এক এ ঐ ই ইহা ইত্যাদি উনি উপর উপরে উচিত ও ওই ওর ওরা ওঁর ওঁরা ওকে ওদের ওঁদের ওখানে কত কবে করতে কয়েক কয়েকটি করবে করলেন করার কারও করা করি করিয়ে করার করাই করলে করলেন করিতে করিয়া করেছিলেন করছে করছেন করেছেন করেছে করেন করবেন করায় করে করেই কাছ কাছে কাজে কারণ কিছু কিছুই কিন্তু কিংবা কি কী কেউ কেউই কাউকে কেন কে কোনও কোনো কোন কখনও ক্ষেত্রে খুব গুলি গিয়ে গিয়েছে গেছে গেল গেলে গোটা চলে ছাড়া ছাড়াও ছিলেন ছিল জন্য জানা ঠিক তিনি তিনঐ তিনিও তখন তবে তবু তাঁদের তাঁাহারা তাঁরা তাঁর তাঁকে তাই তেমন তাকে তাহা তাহাতে তাহার তাদের তারপর তারা তারৈ তার তাহলে তিনি তা তাও তাতে তো তত তুমি তোমার তথা থাকে থাকা থাকায় থেকে থেকেও থাকবে থাকেন থাকবেন থেকেই দিকে দিতে দিয়ে দিয়েছে দিয়েছেন দিলেন দু দুটি দুটো দেয় দেওয়া দেওয়ার দেখা দেখে দেখতে দ্বারা ধরে ধরা নয় নানা না নাকি নাগাদ নিতে নিজে নিজেই নিজের নিজেদের নিয়ে নেওয়া নেওয়ার নেই নাই পক্ষে পর্যন্ত পাওয়া পারেন পারি পারে পরে পরেই পরেও পর পেয়ে প্রতি প্রভৃতি প্রায় ফের ফলে ফিরে ব্যবহার বলতে বললেন বলেছেন বলল বলা বলেন বলে বহু বসে বার বা বিনা বরং বদলে বাদে বার বিশেষ বিভিন্ন বিষয়টি ব্যবহার ব্যাপারে ভাবে ভাবেই মধ্যে মধ্যেই মধ্যেও মধ্যভাগে মাধ্যমে মাত্র মতো মতোই মোটেই যখন যদি যদিও যাবে যায় যাকে যাওয়া যাওয়ার যত যতটা যা যার যারা যাঁর যাঁরা যাদের যান যাচ্ছে যেতে যাতে যেন যেমন যেখানে যিনি যে রেখে রাখা রয়েছে রকম শুধু সঙ্গে সঙ্গেও সমস্ত সব সবার সহ সুতরাং সহিত সেই সেটা সেটি সেটাই সেটাও সম্প্রতি সেখান সেখানে সে স্পষ্ট স্বয়ং হইতে হইবে হৈলে হইয়া হচ্ছে হত হতে হতেই হবে হবেন হয়েছিল হয়েছে হয়েছেন হয়ে হয়নি হয় হয়েই হয়তো হল হলে হলেই হলেও হলো হিসাবে হওয়া হওয়ার হওয়ায় হন হোক জন জনকে জনের জানতে জানায় জানিয়ে জানানো জানিয়েছে জন্য জন্যওজে জে বেশ দেন তুলে ছিলেন চান চায় চেয়ে মোট যথেষ্ট টি অতএব অথচ অথবা অনুযায়ী অনেক অনেকে অনেকেই অন্তত অন্য অবধি অবশ্য অর্থাত আই আগামী আগে আগেই আছে আজ আদ্যভাগে আপনার আপনি আবার আমরা আমাকে আমাদের আমার আমি আর আরও ই ইত্যাদি ইহা উচিত উত্তর উনি উপর উপরে এ এঁদের এঁরা এই একই একটি একবার একে এক্ এখন এখনও এখানে এখানেই এটা এটাই এটি এত এতটাই এতে এদের এব এবং এবার এমন এমনকী এমনি এর এরা এল এস এসে ঐ ও ওঁদের ওঁর ওঁরা ওই ওকে ওখানে ওদের ওর ওরা কখনও কত কবে কমনে কয়েক কয়েকটি করছে করছেন করতে করবে করবেন করলে করলেন করা করাই করায় করার করি করিতে করিয়া করিয়ে থাকবেন থাকা থাকায় থাকে থাকেন থেকে থেকেই থেকেও দিকে দিতে দিন দিয়ে দিয়েছে দিয়েছেন দিলেন দু দুই দুটি দুটো দেওয়া দেওয়ার দেওয়া দেখতে দেখা দেখে দেন দেয় দ্বারা ধরা ধরে ধামার নতুন নয় না নাই নাকি নাগাদ নানা নিজে নিজেই নিজেদের নিজের নিতে নিয়ে নিয়ে নেই নেওয়া নেওয়ার নেওয়া নয় পক্ষে পর পরে পরেই পরেও পর্যন্ত পাওয়া পাচ পারি পারে পারেন পি পেয়ে পেয়্র্ প্রতি প্রথম প্রভৃতি প্রযন্ত প্রাথমিক প্রায় প্রায় ফলে ফিরে ফের বক্তব্য বদলে বন বরং বলতে বলল বললেন বলা বলে বলেছেন বলেন বসে হু বা বাদে বার বি বিনা বিভিন্ন বিশেষ বিষয়টি  বেশ বেশি ব্যবহার ব্যাপারে ভাবে ভাবেই মতো মতোই মধ্যভাগে মধ্যে মধ্যেই মধ্যেও মনে মাত্র মাধ্যমে মোট মোটেই যখন যত যতটা যথেষ্ট যদি যদিও যা যাঁর যাঁরা যাওয়া যাওয়ার যাওয়া যাকে যাচ্ছে যাতে যাদের যান যাবে যায় যার যারা যিনি যে যেখানে যেতে যেন যেমন র রকম রয়েছে রাখা রেখে লক্ষ শুধু শুরু সঙ্গে সঙ্গেও সব সবার সমস্ত সম্প্রতি সহ সহিত সাধারণ সামনে সি সুতরাং সে সেই নয় না হয় হয়ে হয়েছে হয়েছিলেন । , - ; ঃ মিথ্যা শুরু"

In [ ]:
stopwords = []

for word in stopwords_1:
  if word not in stopwords:
    stopwords.append(word)

for word in stopwords_2.split():
  if word not in stopwords:
    stopwords.append(word)

len(stopwords)

445

In [ ]:
def preprocess_BN(text, stop_words=None):
  #text = text.split()
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r'\<a href', ' ', text)
  text = re.sub(r'&amp;', '', text) 
  text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
  text = re.sub(r'<br />', ' ', text)
  text = re.sub(r'\'', ' ', text)
  text = re.sub(r'।', ' ', text)
  text = re.sub(r'[‘’]', ' ', text)
  text = re.sub(r'[০১২৩৪৫৬৭৮৯123456789]', ' ', text)
  text = re.sub(r'[a-zA-Z]', ' ', text)
  #text = " ".join(text)
  
  #print("In PP (before stopwords): {}".format(text))
  '''text = text.split(' ')
  text = [word for word in text if word not in stopwords]'''
  
  tokenizer = Tokenizers()
  stemmer = RafiStemmer()

  #stop_words = load_stop_word()
  

  doc_token = []

  if isinstance(text, str):
      for token in tokenizer.bn_word_tokenizer(text):
          if token not in stop_words and len(token) >= 3:
              stemmed_token = stemmer.stem_word(token)
              if len(stemmed_token) >= 2 and stemmed_token not in stop_words:
                  doc_token.append(stemmed_token)


  #text = remove_stopwords(text, stopwords=stopwords)

  '''if isinstance(text, str):
    text = text.split()
  text = stemming_docs(text)'''
  #print(text)
  return " ".join(doc_token)

In [ ]:
# remove symbols, punctuations, and stopwords

pp_bangla_news_list = []
for bangla_news in bangla_news_list:
  #print("Type: {}".format(type(str(bangla_news))))
  #print("Before PP: {}".format(bangla_news))
  pp_bangla_news_list.append(preprocess_BN(bangla_news, stopwords))
  #print("After PP: {}\n\n".format(pp_bangla_news_list[-1]))

In [ ]:
%cache bangla_news_list = bangla_news_list

Loading cached value for variable 'bangla_news_list'. Time since caching: 3 days, 13:23:26.979370


In [ ]:
%cache pp_bangla_news_list = pp_bangla_news_list

Loading cached value for variable 'pp_bangla_news_list'. Time since caching: 3 days, 22:45:35.748597


In [ ]:
len(pp_bangla_news_list)

373310

In [ ]:
for i in range(500,505):
  print("Before PP: {}".format(bangla_news_list[i]))
  print("After PP: {}\n\n".format(pp_bangla_news_list[i]))

In [ ]:
import regex 

def custom_analyzer(text):
    words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
    for w in words:
        yield w

# LDA Model

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
'''import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt'''
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
def preprocess_BN_wo_stem(text, stop_words=None):
  #text = text.split()
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r'\<a href', ' ', text)
  text = re.sub(r'&amp;', '', text) 
  text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
  text = re.sub(r'<br />', ' ', text)
  text = re.sub(r'\'', ' ', text)
  text = re.sub(r'।', ' ', text)
  text = re.sub(r'[‘’]', ' ', text)
  text = re.sub(r'[০১২৩৪৫৬৭৮৯123456789]', ' ', text)
  text = re.sub(r'[a-zA-Z]', ' ', text)
  #text = " ".join(text)
  
  #print("In PP (before stopwords): {}".format(text))
  '''text = text.split(' ')
  text = [word for word in text if word not in stopwords]'''
  
  tokenizer = Tokenizers()
  stemmer = RafiStemmer()

  #stop_words = load_stop_word()
  

  doc_token = []

  if isinstance(text, str):
    for token in tokenizer.bn_word_tokenizer(text):
      if token not in stop_words and len(token) >= 3:
        doc_token.append(token)


  #text = remove_stopwords(text, stopwords=stopwords)

  '''if isinstance(text, str):
    text = text.split()
  text = stemming_docs(text)'''
  #print(text)
  return " ".join(doc_token)

In [ ]:
preprocess_BN("পানির অপর নাম জীবন। পানি ছাড়া যেকোন প্রাণীর জীবন ধারণ অসম্ভব। তবে প্রয়োজনের বেশি পানি খেলে তৈরি হতে পারে সমস্যা।", stop_words=stopwords)

'পানির অপর নাম জীবন পানি যেকোন প্রাণীর জীবন ধারণ অসম্ভব প্রয়োজন পানি খেল তৈরি সমস্যা'

In [ ]:
tokenized_corpus = []
for doc in pp_bangla_news_list[:100000]:
  tokenized_corpus.append(list(custom_analyzer(doc)))

In [ ]:
tokenized_corpus = []
tokenized_corpus.append(list(custom_analyzer('পানি অপর নাম জীবন পানি যেকোন প্রাণীর জীবন ধারণ অসম্ভব প্রয়োজন বেশি পানি খেল সমস্যা')))

In [ ]:
tokenized_corpus

[['পানি',
  'অপর',
  'নাম',
  'জীবন',
  'পানি',
  'যেকোন',
  'প্রাণীর',
  'জীবন',
  'ধারণ',
  'অসম্ভব',
  'প্রয়োজন',
  'বেশি',
  'পানি',
  'খেল',
  'সমস্যা']]

In [ ]:
id2word[2]

'খেল'

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(tokenized_corpus)

# Create Corpus
texts = tokenized_corpus

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 3), (14, 2), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 2), (60, 4), (61, 1), (62, 3), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 4), (70, 4), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 8), (86, 1), (87, 2), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 3), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 2), (104, 2), (105, 1), (106, 3), (107, 2), (108, 1), (109, 1), (110, 1)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           eval_every=10,
                                           chunksize=10000,
                                           passes=150,
                                           per_word_topics=True,
                                           workers=3)

In [ ]:
from gensim.test.utils import datapath

# Save model to disk.

temp_file = datapath("/content/drive/My Drive/THESIS/LDA/ldamodel/lda_model")

lda_model.save(temp_file)

In [ ]:
# Load a potentially pretrained model from disk.
from gensim.test.utils import datapath
temp_file = datapath("/content/drive/My Drive/THESIS/LDA/ldamodel/lda_model")

lda_model = gensim.models.LdaModel.load(temp_file)

In [ ]:
NUM_TOPICS = 20

for idx in range(NUM_TOPICS):
        # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))

Topic #0: 0.016*"মানুষ" + 0.012*"বছর" + 0.010*"নারী" + 0.010*"শিশু" + 0.007*"পড়" + 0.006*"সম" + 0.006*"নাম" + 0.005*"মা" + 0.005*"গবেষণা" + 0.004*"সাল"
Topic #1: 0.023*"মামলা" + 0.021*"পুলিশ" + 0.015*"অভিযোগ" + 0.012*"বিরুদ্ধ" + 0.010*"আসামি" + 0.010*"আদালত" + 0.009*"মামল" + 0.008*"তদন্ত" + 0.008*"গ্রেপ্ত" + 0.008*"আদাল"
Topic #2: 0.015*"পুলিশ" + 0.012*"হামলা" + 0.010*"ঘটনা" + 0.007*"হামল" + 0.007*"দা" + 0.006*"বিক্ষোভ" + 0.006*"নিহত" + 0.006*"বিরুদ্ধ" + 0.006*"পাকিস্তান" + 0.005*"অভিযোগ"
Topic #3: 0.010*"খাব" + 0.009*"শরীর" + 0.009*"পানি" + 0.008*"রাখ" + 0.007*"সমস্যা" + 0.006*"সম" + 0.006*"করুন" + 0.006*"ভালো" + 0.006*"চুল" + 0.006*"ত্বক"
Topic #4: 0.017*"ট্রাম্প" + 0.012*"প্রেসিডেন্ট" + 0.012*"যুক্তরাষ্ট্র" + 0.011*"প্রতিষ্ঠান" + 0.011*"মার্কিন" + 0.006*"তথ্য" + 0.006*"বছর" + 0.006*"অ্যাপ" + 0.006*"সাল" + 0.006*"ফোন"
Topic #5: 0.013*"ভালো" + 0.012*"কথা" + 0.008*"সম" + 0.007*"জীবন" + 0.007*"আপনা" + 0.006*"মানুষ" + 0.005*"আস" + 0.005*"করুন" + 0.005*"শুভ" + 0.004*"থাক"
Topic #6: 0.027*

In [ ]:
bangla_news_list[128765]

' রোববার দিবাগত রাত পৌনে ২টার দিকে টিএমএসএস হাসপাতালে মাহাদী নামের ওই শিশুর মৃত্যু হয় বলে হাসপাতালের জেনারেল অ্যাডমিন ও করোনা সংক্রান্ত মুখপাত্র আব্দুর রহিম রুবেল জানান। মাহাদী বগুড়া নুনগোলা ডিগ্রী কলেজের প্রভাষক শহরের দক্ষিণ ঠনঠনিয়া এলাকার আব্দুল ওহাবের ছেলে। পরিবারের বরাতে রুবেল বলেন, ব্লাড ক্যান্সারে আক্রান্ত মাহাদী ঢাকার একটি হাসপাতালে ভর্তি ছিল। গত ১৪ আগস্ট নমুনা পরীক্ষায় তার কোভিড-১৯ পজিটিভ শনাক্ত হয়। এরপর শিশুটির শারীরিক অবস্থার অবনতি হলে রোববার রাত সাড়ে ১২টার দিকে টিএমএসএস হাসপাতালে ভর্তির দেড় ঘণ্টা পরে সে মারা যায়। কোয়ান্টাম ফাউন্ডেশন বগুড়া শাখার সমন্বয়ক মিজানুর রহমান জানান, তাদের স্বেচ্ছাসেবীদের একটি দল স্বাস্থ্যবিধি মেনে হাসপাতাল চত্ত্বরে জানাজা শেষে শিশুটির লাশ পরিবারের কাছে হস্তান্তর করেছে।'

In [ ]:
bow_vector = id2word.doc2bow(list(custom_analyzer(preprocess_BN(bangla_news_list[128765], stop_words=stopwords))))

In [ ]:
for topic, prob in sorted(lda_model.get_document_topics(bow_vector), key=lambda tup: -1*tup[1]):
    print(topic_num_to_word[topic], prob)

In [ ]:
tokenized_corpus = []
for doc in pp_bangla_news_list[:150000]:
  tokenized_corpus.append(list(custom_analyzer(doc)))

# Create Dictionary
id2word = corpora.Dictionary(tokenized_corpus)

# Create Corpus
texts = tokenized_corpus

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
pp_bangla_news_list[128765]

'রোবব দিবাগত রাত পৌন টিএমএসএস হাসপাতাল মাহাদী নাম শিশুর মৃত্যু হাসপাতাল জেনার অ্যাডমিন করোনা সংক্রান্ত মুখপাত্র আব্দুর রহিম রুব জানান মাহাদী বগুড়া নুনগোলা ডিগ্রী কলেজ প্রভাষক শহর দক্ষিণ ঠনঠনিয়া এলাক আব্দুল ওহাব ছেল পরিবার বরা রুব ব্লাড ক্যান্সার আক্রান্ত মাহাদী ঢাক হাসপাতাল ভর্তি আগস্ট নমুনা পরীক্ষা কোভিড পজিটিভ শনাক্ত এরপর শিশু শারীরিক অবস্থ অবনতি রোবব রাত সাড় টিএমএসএস হাসপাতাল ভর্তির দেড় ঘণ্ মা কোয়ান্টাম ফাউন্ডেশন বগুড়া শাখ সমন্বয়ক মিজানুর রহমান জানান স্বেচ্ছাসেবী স্বাস্থ্যবিধি মেন হাসপাতাল চত্ত্বর জানাজা শেষ শিশু লাশ পরিবার হস্তান্তর'

In [ ]:
#get_document_topics = lda_model.get_document_topics(corpus)
print(sorted(get_document_topics[128765], key=lambda tup: -1*tup[1]))

[(14, 0.20968664), (5, 0.1475669), (12, 0.12959209), (9, 0.09795202), (16, 0.07185722), (8, 0.06567942), (4, 0.06442191), (1, 0.062812336), (6, 0.048931457), (0, 0.041497312), (10, 0.022887586), (13, 0.017260324), (3, 0.016261067)]


In [ ]:
lda_model.show_topics(num_topics=20, num_words=10)

[(0,
  '0.016*"মানুষ" + 0.012*"বছর" + 0.010*"নারী" + 0.010*"শিশু" + 0.007*"পড়" + 0.006*"সম" + 0.006*"নাম" + 0.005*"মা" + 0.005*"গবেষণা" + 0.004*"সাল"'),
 (1,
  '0.023*"মামলা" + 0.021*"পুলিশ" + 0.015*"অভিযোগ" + 0.012*"বিরুদ্ধ" + 0.010*"আসামি" + 0.010*"আদালত" + 0.009*"মামল" + 0.008*"তদন্ত" + 0.008*"গ্রেপ্ত" + 0.008*"আদাল"'),
 (2,
  '0.015*"পুলিশ" + 0.012*"হামলা" + 0.010*"ঘটনা" + 0.007*"হামল" + 0.007*"দা" + 0.006*"বিক্ষোভ" + 0.006*"নিহত" + 0.006*"বিরুদ্ধ" + 0.006*"পাকিস্তান" + 0.005*"অভিযোগ"'),
 (3,
  '0.010*"খাব" + 0.009*"শরীর" + 0.009*"পানি" + 0.008*"রাখ" + 0.007*"সমস্যা" + 0.006*"সম" + 0.006*"করুন" + 0.006*"ভালো" + 0.006*"চুল" + 0.006*"ত্বক"'),
 (4,
  '0.017*"ট্রাম্প" + 0.012*"প্রেসিডেন্ট" + 0.012*"যুক্তরাষ্ট্র" + 0.011*"প্রতিষ্ঠান" + 0.011*"মার্কিন" + 0.006*"তথ্য" + 0.006*"বছর" + 0.006*"অ্যাপ" + 0.006*"সাল" + 0.006*"ফোন"'),
 (5,
  '0.013*"ভালো" + 0.012*"কথা" + 0.008*"সম" + 0.007*"জীবন" + 0.007*"আপনা" + 0.006*"মানুষ" + 0.005*"আস" + 0.005*"করুন" + 0.005*"শুভ" + 0.004*"থাক"'),
 (6,
  '0.

In [ ]:
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

In [ ]:
lda_model.print_topics(num_topics=20, num_words=10)

In [ ]:
topics_prod = []

for idx in range(20):
    topics_prod.append([word for word, prob in lda_model.show_topic(idx, 10)])

topics_prod

[['মানুষ', 'বছর', 'নারী', 'শিশু', 'পড়', 'সম', 'নাম', 'মা', 'গবেষণা', 'সাল'],
 ['মামলা',
  'পুলিশ',
  'অভিযোগ',
  'বিরুদ্ধ',
  'আসামি',
  'আদালত',
  'মামল',
  'তদন্ত',
  'গ্রেপ্ত',
  'আদাল'],
 ['পুলিশ',
  'হামলা',
  'ঘটনা',
  'হামল',
  'দা',
  'বিক্ষোভ',
  'নিহত',
  'বিরুদ্ধ',
  'পাকিস্তান',
  'অভিযোগ'],
 ['খাব', 'শরীর', 'পানি', 'রাখ', 'সমস্যা', 'সম', 'করুন', 'ভালো', 'চুল', 'ত্বক'],
 ['ট্রাম্প',
  'প্রেসিডেন্ট',
  'যুক্তরাষ্ট্র',
  'প্রতিষ্ঠান',
  'মার্কিন',
  'তথ্য',
  'বছর',
  'অ্যাপ',
  'সাল',
  'ফোন'],
 ['ভালো', 'কথা', 'সম', 'জীবন', 'আপনা', 'মানুষ', 'আস', 'করুন', 'শুভ', 'থাক'],
 ['করোনা',
  'করোনাভাইরাস',
  'আক্রান্ত',
  'হাসপাতাল',
  'দেশ',
  'স্বাস্থ্য',
  'রোগী',
  'মৃত্যু',
  'সংক্রমণ',
  'সংখ্যা'],
 ['শিক্ষার্থী',
  'বিভাগ',
  'শিক্ষক',
  'পরীক্ষা',
  'বিশ্ববিদ্যালয়',
  'শিক্ষা',
  'আবেদন',
  'কমি',
  'বিশ্ববিদ্যাল',
  'কলেজ'],
 ['দেশ',
  'চীন',
  'ভারত',
  'যুক্তরাষ্ট্র',
  'বাংলাদেশ',
  'বিমান',
  'রোহিঙ্গা',
  'সৌদি',
  'ইরান',
  'আন্তর্জাতিক'],
 ['ম্যাচ',
  'গোল',
  'দল',
 

In [ ]:
# Compute Coherence Score with Model
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_corpus, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  -1.8642676094659554


In [ ]:
tokenized_corpus_test = []
for doc in pp_bangla_news_list[100000:150000]:
  tokenized_corpus_test.append(list(custom_analyzer(doc)))

In [ ]:
# Compute Coherence Score with Topics
coherence_model_lda = CoherenceModel(topics=topics_prod, texts=tokenized_corpus, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  -1.6811436125515968
